In [1]:
!pip install libspatialindex-dev
!pip install rtree
!pip install geopandas

ERROR: Could not find a version that satisfies the requirement libspatialindex-dev (from versions: none)
ERROR: No matching distribution found for libspatialindex-dev
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 115.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 166.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.5 MB/s eta 0:00:00


In [2]:
import os
import sys

In [3]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('input_nominatim.csv')
df

,location,doc_num
0,- Region 1: Baakenhafen 4,22_2088
1,Baakenhafen 4,22_2088
2,Gymnasium Oldenfelde 5,22_455
3,Elbkinder Grundschule 6,22_455
4,Berlin Saimen 1,22_5213
...,...,...
144,Hamburger Frauenhäuser,22_8489
145,Spielplätze and Grünanlagen,22_8489
146,Feuerwehr,22_8489
147,Klütjenfelder Lieger,22_8489


In [4]:
#geocoded deepseek remaining rows (not given coordinates by mapbox) with nominatim
# geocode the location with Nominatim API and save result to csv
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point



def geocode_location(location_name):
    """
    Geocodes a location name using the Nominatim API.

    Args:
        location_name: The name of the location to geocode.

    Returns:
        A dictionary containing the latitude and longitude, or None if not found.
    """
    base_url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': location_name,
        'format': 'jsonv2',
        'limit': 1 # Limit to one result
    }
    headers = {'User-Agent': 'My Geocoding Application'}  # Replace with your app description

    response = requests.get(base_url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if data:
            result = data[0]
            return {
                'latitude': float(result['lat']),
                'longitude': float(result['lon'])
            }
        else:
            print(f"No results found for '{location_name}'")
            return None
    else:
        print(f"API request failed with status code: {response.status_code}")
        return None


# May need to change 'location', depending on column name
geocoded_data = []
notcoded_data = []
for _, row in df.iterrows():
    location_name = row['location']
    # Call the geocoding function, not the DataFrame
    coordinates = geocode_location(location_name)
    if coordinates:
        geocoded_data.append({
            'location_name': location_name,
            'latitude': coordinates['latitude'],
            'longitude': coordinates['longitude'],
            'doc_num': row['doc_num']
        })
    else:
        notcoded_data.append({
            'location_name': location_name,
            'latitude': None,
            'longitude': None,
            'doc_num': row['doc_num']
        })

# Create a new DataFrame from the geocoded data
geocoded_df = pd.DataFrame(geocoded_data)
geocoded_df

No results found for '- Region 1: Baakenhafen 4'
No results found for 'Gymnasium Oldenfelde 5'
No results found for 'Elbkinder Grundschule 6'
No results found for 'Berlin Saimen 1'
No results found for 'Poberweg 34'
No results found for 'Prusselstraße 5'
No results found for 'Otterbrock 17/19'
No results found for 'Dauerbreiten 13'
No results found for 'Mührmehammaberg 75'
No results found for 'Münchmannstraße 61'
No results found for 'Münchmannstraße 60'
No results found for 'Lohstapel 6'
No results found for 'Lohstapel 10'
No results found for 'Lohstapel 100'
No results found for 'Fränstraße 50'
No results found for 'Dornentstraße 26'
No results found for 'Brassenhof 3 (27)'
No results found for 'Shigterreihe 1'
No results found for 'Münchmannstraße 125'
No results found for 'Strassebrunde 36'
No results found for 'Strassebrunde 38'
No results found for 'Vogelhaltsnadeln 120'
No results found for 'Grundsportlade 33'
No results found for 'Hobbenkontrolle 51'
No results found for 'Hobb

,location_name,latitude,longitude,doc_num
0,Baakenhafen 4,53.538849,10.010850,22_2088
1,Borgsweg 176,51.112808,6.050391,22_5213
2,Hebei 26,40.755958,114.884149,22_5213
3,- Umweltwache F32,53.511871,9.967211,22_5589
4,Deutsch-französisches Gymnasium,49.229810,7.016921,22_2088
5,Wandsbek Bezirk,53.574536,10.082354,22_2592
6,Harburg Bezirk,53.550340,10.012087,22_2592
7,Bergedorf Bezirk,53.498601,10.181979,22_2592
8,Liliencronpark,53.600050,10.154771,22_2957
9,Planetarium),53.597313,10.008911,22_2957


In [5]:
geocoded_df.to_csv("geocoded_deepseek_nominatim.csv")

In [7]:
no_coordinate = pd.DataFrame(notcoded_data)
no_coordinate

,location_name,latitude,longitude,doc_num
0,- Region 1: Baakenhafen 4,None,None,22_2088
1,Gymnasium Oldenfelde 5,None,None,22_455
2,Elbkinder Grundschule 6,None,None,22_455
3,Berlin Saimen 1,None,None,22_5213
4,Poberweg 34,None,None,22_5213
...,...,...,...,...
87,- Nordkirche,None,None,22_812
88,Hamburger Frauenhäuser,None,None,22_8489
89,Spielplätze and Grünanlagen,None,None,22_8489
90,Klütjenfelder Lieger,None,None,22_8489


This section is only for Claude 
Split location_name with "/" in between, and save result to csv file containing location_name and doc_num

In [14]:
# this section is only for claude / llama, most of the case no need to run
# Split location names and create a new DataFrame
new_rows = []
for _, row in df.iterrows():
    new_rows.append({'location': row['location'].strip(), 'doc_num': row['file_number'][:-2]})
  #locations = row['location'].split('/')
  #for location in locations:
    

new_df = pd.DataFrame(new_rows)

# Remove duplicate location names for each doc_num
new_df = new_df.drop_duplicates(subset=['location', 'doc_num'])

# Save the result to a new CSV file
new_df.to_csv('location_llama31_70b_extract.csv', index=False)

In [12]:
new_df

,location,doc_num
0,Stubbenhof 15,22_1492
1,Maretstraße,22_1492
2,S-Bahn-Station Hamburg-Harburg,22_1695
3,Neugrabener Bahnhofstraße 35,22_1695
4,Neugrabener Bahnhofstraße 2,22_1695
...,...,...
1018,Lurup,22_8489
1019,Anna-Susanna-Stieg,22_8565
1020,Schnelsen,22_8565
1021,Burgwedel,22_8565


In [33]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('human_label_valid.csv')
# Split location names and create a new DataFrame
new_rows = []
for _, row in df.iterrows():
  locations = row['location_name'].split('/')
  for location in locations:
    new_rows.append({'location_name': location.strip(), 'doc_num': row['doc_num']})

new_df = pd.DataFrame(new_rows)

# Remove duplicate location names for each doc_num
new_df = new_df.drop_duplicates(subset=['location_name', 'doc_num'])

# Save the result to a new CSV file
new_df.to_csv('human_label_valid_extract.csv', index=False)

Geocoding individual location table with Nominatim

In [17]:
import pandas as pd

# Replace with path to location table Google Drive
#file_path = '/content/drive/MyDrive/LIG/last task/location_claude_sonnet.csv'
file_path = 'location_llama31_70b_extract.csv'
#geocode_df = pd.read_csv(file_path)
geocode_df = new_df
print(geocode_df.head())
print(len(geocode_df))

                         location  doc_num
0                   Stubbenhof 15  22_1492
1                     Maretstraße  22_1492
2  S-Bahn-Station Hamburg-Harburg  22_1695
3    Neugrabener Bahnhofstraße 35  22_1695
4     Neugrabener Bahnhofstraße 2  22_1695
1023


In [20]:
# geocode the location with Nominatim API and save result to csv
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point



def geocode_location(location_name):
    """
    Geocodes a location name using the Nominatim API.

    Args:
        location_name: The name of the location to geocode.

    Returns:
        A dictionary containing the latitude and longitude, or None if not found.
    """
    base_url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': location_name,
        'format': 'jsonv2',
        'limit': 1 # Limit to one result
    }
    headers = {'User-Agent': 'My Geocoding Application'}  # Replace with your app description

    response = requests.get(base_url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if data:
            result = data[0]
            return {
                'latitude': float(result['lat']),
                'longitude': float(result['lon'])
            }
        else:
            print(f"No results found for '{location_name}'")
            return None
    else:
        print(f"API request failed with status code: {response.status_code}")
        return None


# May need to change 'location', depending on column name
geocoded_data = []
notcoded_data = []
for _, row in geocode_df.iterrows():
    location_name = row['location']
    # Call the geocoding function, not the DataFrame
    coordinates = geocode_location(location_name)
    if coordinates:
        geocoded_data.append({
            'location_name': location_name,
            'latitude': coordinates['latitude'],
            'longitude': coordinates['longitude'],
            'doc_num': row['doc_num']
        })
    else:
        notcoded_data.append({
            'location_name': location_name,
            'latitude': None,
            'longitude': None,
            'doc_num': row['doc_num']
        })

# Create a new DataFrame from the geocoded data
geocoded_df = pd.DataFrame(geocoded_data)

# Assuming your CSV has columns named 'latitude' and 'longitude'
# Create a geometry column from the coordinates
geocoded_df = pd.DataFrame(geocoded_data)

# Filter out rows with missing coordinates before creating Point objects
#geocoded_df_valid = geocoded_df.dropna(subset=['latitude', 'longitude'])




No results found for 'Rehrstieg-Siedlung'
No results found for 'Gumbrechtstraße 14'
No results found for 'Am Tröndel 1'
No results found for 'Lese-Café Stadtpark'
No results found for 'Süderelbischen Verbindungsweg 19'
No results found for 'Carl-von-Ossietzky-Gymnasiums'
No results found for 'Großenottastraße 9'
No results found for 'Slomanstieg 1/3'
No results found for 'Struenseecampus'
No results found for 'Pipinskamp 8'
No results found for 'Neue Grundschule in Bergedorf'
No results found for 'Stadtteilschule in Bergedorf'
No results found for 'Neue Grundschule Eschenweg'
No results found for 'Neues Gymnasium Eilbektal'
No results found for 'Neues Gymnasium Grellkamp'
No results found for 'Heinrich-Helbing-Schule'
No results found for 'Schule Bellerstaedtweg'
No results found for 'Schule Telmannstraße'
No results found for 'Schule Sinstorf'
No results found for 'An der Seebeck'
No results found for 'Schimmelmannstraße Höhe Holstenhofweg'
No results found for 'Volksdorfer Marktplatz

In [21]:
geometry = [Point(xy) for xy in zip(geocoded_df['longitude'], geocoded_df['latitude'])]
gdf = gpd.GeoDataFrame(geocoded_df, geometry=geometry, crs='EPSG:4326')

# Filter locations within Hamburg's bounding box (approximate)
hamburg_bbox = (9.734, 53.392, 10.325, 53.754)  # (min_lon, min_lat, max_lon, max_lat)
hamburg_gdf = gdf.cx[hamburg_bbox[0]:hamburg_bbox[2], hamburg_bbox[1]:hamburg_bbox[3]]

In [22]:
geocoded_df

,location_name,latitude,longitude,doc_num
0,Stubbenhof 15,53.479245,9.877513,22_1492
1,Maretstraße,53.453442,9.982409,22_1492
2,S-Bahn-Station Hamburg-Harburg,53.474072,9.852487,22_1695
3,Neugrabener Bahnhofstraße 35,53.468970,9.854685,22_1695
4,Neugrabener Bahnhofstraße 2,53.473208,9.854287,22_1695
...,...,...,...,...
901,Lurup,53.592952,9.874564,22_8489
902,Anna-Susanna-Stieg,53.642806,9.916200,22_8565
903,Schnelsen,53.637400,9.918831,22_8565
904,Burgwedel,52.500000,9.866670,22_8565


In [23]:
notcoded_df = pd.DataFrame(notcoded_data)
notcoded_df

,location_name,latitude,longitude,doc_num
0,Rehrstieg-Siedlung,None,None,22_1695
1,Gumbrechtstraße 14,None,None,22_1695
2,Am Tröndel 1,None,None,22_1695
3,Lese-Café Stadtpark,None,None,22_1872
4,Süderelbischen Verbindungsweg 19,None,None,22_2088
...,...,...,...,...
112,Straße Beim Tr trespass 2,None,None,22_8489
113,Hass beforeSendende 80,None,None,22_8489
114,Eichbaum-Strecke,None,None,22_8489
115,Mittelbert 9,None,None,22_8489


In [24]:
# Save the geocoded data to a CSV file
geocoded_df.to_csv('location_llama31_70b.csv', index=False)
notcoded_df.to_csv('notcoded_llama31_70b.csv', index=False)

In [25]:
# save the result to geojson

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load the CSV file with coordinates
file_path = 'location_llama31_70b.csv'

df = pd.read_csv(file_path)

# Save the filtered GeoDataFrame to a csv file
hamburg_gdf.to_csv('location_clean_table_llama31_nominatim.csv', index=False)


In [26]:
hamburg_gdf.to_file('location_clean_table_llama31_nominatim.geojson', driver='GeoJSON')
#hamburg_gdf.to_csv('location_clean_table_llama31_nominatim.csv', index=False)
print("GeoJSON file created.")

GeoJSON file created.


Geocoding locations from multiple PDF in one location_clean_table
Spatial join human_label and prompt_response

In [50]:
# Read human_label; containing all location_name from human label for 24 doc
import geopandas as gpd
import pandas as pd

# Replace with the actual path to csv file in Google Drive
file_path2 = 'human_label_valid_extract.csv'
geocode_df2 = pd.read_csv(file_path2)

# Assuming 'geocode_df' is your DataFrame and 'geocode_location' is your function
geocoded_data2 = []
invalid_data2 = []
for _, row in geocode_df2.iterrows():
    location_name = row['location_name']
    doc_num = row['doc_num']
    coordinates = geocode_location(location_name)
    if coordinates:
        geocoded_data2.append({
            'location_name': location_name,
            'doc_num': doc_num,
            'latitude': coordinates['latitude'],
            'longitude': coordinates['longitude']
        })
    else:
        invalid_data2.append({
            'location_name': location_name,
            'latitude': None,
            'longitude': None
        })


# Create a new DataFrame from the geocoded data
geocoded_df2 = pd.DataFrame(geocoded_data2)
invalid_df2 = pd.DataFrame(invalid_data2)

# Filter out rows with missing coordinates before creating Point objects
#geocoded_df2_valid = geocoded_df2.dropna(subset=['latitude', 'longitude'])

geometry = [Point(xy) for xy in zip(geocoded_df2_valid['longitude'], geocoded_df2_valid['latitude'])]
gdf2 = gpd.GeoDataFrame(geocoded_df2_valid, geometry=geometry, crs='EPSG:4326')

# Filter locations within Hamburg's bounding box (approximate)
hamburg_bbox = (9.734, 53.392, 10.325, 53.754)  # (min_lon, min_lat, max_lon, max_lat)
hamburg_gdf = gdf.cx[hamburg_bbox[0]:hamburg_bbox[2], hamburg_bbox[1]:hamburg_bbox[3]]


# Save the geocoded data to a CSV file
# Save to geojson
# Load the CSV file with coordinates
file_path = 'human_label_valid_extract_nominatim.csv'
df = pd.read_csv(file_path)

# Filter out rows with missing coordinates before creating Point objects
df_valid = df.dropna(subset=['latitude', 'longitude'])

# Create a geometry column from the coordinates
geometry = [Point(xy) for xy in zip(df_valid['longitude'], df_valid['latitude'])]
gdf = gpd.GeoDataFrame(df_valid, geometry=geometry, crs='EPSG:4326')

# Save the GeoDataFrame to a GeoJSON file
#gdf.to_file('/content/drive/MyDrive/LIG/last task/0920/human_label_valid_extract_nominatim.geojson', driver='GeoJSON')
gdf.to_file('human_label_valid_extract_nominatim.geojson', driver='GeoJSON')
print("GeoJSON file created.")
print(gdf)

No results found for 'S-Bahn Neuwiedenthal'
No results found for 'Rehrstieg-Siedlung'
No results found for 'Gumbrechtstraße'
No results found for 'Bleichenbrücke-Nord'
No results found for 'Bleichenbrücke-Süd'
No results found for 'Dietrich-Schreyge-Straße CND 7'
No results found for 'Holtenklinker Straße CND 5'
No results found for 'Moorfuhrtbrücke-Süd'
No results found for 'Neuer Weg CND 2'
No results found for 'Rugenbergerhafen'
No results found for 'Trostbrücke-Nord'
No results found for 'Berufliche Schule Fahrzeugtechnik (BS 16 ehem. G 9)'
No results found for 'Berufliche Schule Holz. Farbe. Textil (BS 25 ehem. G 6)'
No results found for 'Berufliche Schule ITECH Elbinsel Wilhelmsburg (BS 14 ehem. G18)'
No results found for 'Campus Struenseestraße'
No results found for 'Deutsch-Französische Gymnasium'
No results found for 'Elbinselquartiersschule'
No results found for 'Ganztagsgrundschule Am Johannisland'
No results found for 'Grundschule Karlshöhe mit Zweigstellen'
No results foun

In [52]:
print(invalid_df2)

                        location_name latitude longitude
0                S-Bahn Neuwiedenthal     None      None
1                  Rehrstieg-Siedlung     None      None
2                     Gumbrechtstraße     None      None
3                 Bleichenbrücke-Nord     None      None
4                  Bleichenbrücke-Süd     None      None
..                                ...      ...       ...
241         Knoten Pinneberger Straße     None      None
242    Grundschule Anna-Susanna-Stieg     None      None
243  Campus-Stadtteilschule Schnelsen     None      None
244            Campusschule Schnelsen     None      None
245                  Campus Schnelsen     None      None

[246 rows x 3 columns]


In [53]:
invalid_df2.to_csv('no_geocode_human_label.csv', index=False)

In [27]:
# Transform to epsg: 32632 and save as new geojson in meter

import geopandas as gpd

# Load the GeoJSON file
gdf = gpd.read_file('human_label_valid_extract_nominatim_HH.geojson')

# Check the current CRS
print(f"Original CRS: {gdf.crs}")

# Convert to EPSG:32632
gdf = gdf.to_crs('EPSG:32632')

# Check the updated CRS
print(f"Updated CRS: {gdf.crs}")

# Save the transformed GeoDataFrame to a new GeoJSON file
gdf.to_file('human_label_valid_nominatim_projected.geojson', driver='GeoJSON')
print("Transformed GeoJSON file created.")


Original CRS: EPSG:4326
Updated CRS: EPSG:32632
Transformed GeoJSON file created.


In [28]:
# Transform to epsg: 32632 and save as new geojson in meter

import geopandas as gpd

# Load the GeoJSON file
gdf = gpd.read_file('location_clean_table_llama31_nominatim.geojson')

# Check the current CRS
print(f"Original CRS: {gdf.crs}")

# Convert to EPSG:32632
gdf = gdf.to_crs('EPSG:32632')

# Check the updated CRS
print(f"Updated CRS: {gdf.crs}")

# Save the transformed GeoDataFrame to a new GeoJSON file
gdf.to_file("location_clean_table_llama31_nominatim_projected.geojson", driver='GeoJSON')
print("Transformed GeoJSON file created.")

Original CRS: EPSG:4326
Updated CRS: EPSG:32632
Transformed GeoJSON file created.


In [29]:
# Define buffer function

import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import transform
import pyproj

import geopandas as gpd
from shapely.geometry import Polygon

def create_buffer(gdf, radius_meters):
    """
    Creates buffers around geometries in a GeoDataFrame with EPSG:32632 projection.

    Args:
        gdf: The GeoDataFrame containing the geometries.
        radius_meters: The radius of the buffer in meters.

    Returns:
        A GeoDataFrame with the buffered geometries.
    """

    gdf['geometry'] = gdf['geometry'].buffer(radius_meters)
    return gdf

# Load the GeoJSON file
gdf = gpd.read_file('location_clean_table_llama31_nominatim_projected.geojson')

# Create buffer
gdf = create_buffer(gdf, 5)

# Save the GeoDataFrame to a new GeoJSON file
gdf.to_file('location_llama31-70b_nominatim_buffer.geojson', driver='GeoJSON')
print("Buffered GeoJSON file created.")

Buffered GeoJSON file created.


In [30]:
# Define buffer function

import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import transform
import pyproj

import geopandas as gpd
from shapely.geometry import Polygon

def create_buffer(gdf, radius_meters):
    """
    Creates buffers around geometries in a GeoDataFrame with EPSG:32632 projection.

    Args:
        gdf: The GeoDataFrame containing the geometries.
        radius_meters: The radius of the buffer in meters.

    Returns:
        A GeoDataFrame with the buffered geometries.
    """

    gdf['geometry'] = gdf['geometry'].buffer(radius_meters)
    return gdf

# Load the GeoJSON file
gdf = gpd.read_file('human_label_valid_nominatim_projected.geojson')

# Create buffer
gdf = create_buffer(gdf, 5)

# Save the GeoDataFrame to a new GeoJSON file
gdf.to_file('human_label_valid_nominatim_buffer.geojson', driver='GeoJSON')
print("Buffered GeoJSON file created.")

Buffered GeoJSON file created.


In [8]:
import geopandas as gpd
import pandas as pd

In [15]:
# Define spatial join function

def spatial_join_preserve_gdf1_data(geojson1_path, geojson2_path, output_csv_path):
    # Load the two GeoJSON files
    gdf1 = gpd.read_file(geojson1_path)
    gdf2 = gpd.read_file(geojson2_path)

    # Ensure both GeoDataFrames are using the same CRS (coordinate reference system)
    if gdf1.crs != gdf2.crs:
        gdf2 = gdf2.to_crs(gdf1.crs)

    # Rename columns to ensure clarity and avoid overwriting
    gdf1 = gdf1.rename(columns={'location': 'location_1', 'geometry': 'geometry_1', 'doc_num': 'doc_num_1'})
    gdf2 = gdf2.rename(columns={'location': 'location_2', 'geometry': 'geometry_2', 'doc_num': 'doc_num_2'})

    # Prepare to store the result of spatial joins
    spatially_joined = []

    # Loop over each location in gdf1 (base locations)
    for idx1, row1 in gdf1.iterrows():
        buffer_1 = row1['geometry_1']  # gdf1 geometry (buffer)
        found_overlap = False  # Track if we found an overlap for this location in gdf1

        # Loop over each location in gdf2 to check for overlap
        for idx2, row2 in gdf2.iterrows():
            buffer_2 = row2['geometry_2']  # gdf2 geometry (buffer)
            doc_num2 = row2['doc_num_2']
            #print(doc_num2)
            # Condition: Only check for overlap if doc_num_1 == doc_num_2
            if row1['doc_num_1'] == doc_num2:
                # Check if the buffers overlap
                if buffer_1.intersects(buffer_2):
                    found_overlap = True

                    # Add overlapping results to the list, ensuring that gdf1 data is prioritized
                    spatially_joined.append({
                        'doc_num': row1['doc_num_1'],  # doc_num from gdf1
                        'location_1': row1['location_1'],  # From gdf1 (base)
                        'location_2': row2['location_2'],  # From gdf2 (only if overlap)
                        'coordinates_1': buffer_1.centroid.wkt,  # Centroid of geometry from gdf1
                        'coordinates_2': buffer_2.centroid.wkt  # Centroid of geometry from gdf2
                    })
            else:
                # If doc_num_1 != doc_num_2, skip the overlap check
                continue

    # Convert the spatially joined results to a DataFrame
    spatially_joined_df = pd.DataFrame(spatially_joined)

    # Save the result to a CSV file
    spatially_joined_df.to_csv(output_csv_path, index=False)

In [16]:
#spatial join human label with deepseek
spatial_join_preserve_gdf1_data('buffer/buffer_human_label.geojson',
                                'buffer/buffer_deepseek_samples.geojson',
                                'buffer/deepseek_spatial_join.csv')

In [ ]:
spatial_join_preserve_gdf1_data('human_label_valid_nominatim_buffer.geojson',
                                'location_mistral_large_nominatim_buffer.geojson',
                                'mistral-large_spatial_join.csv')